In [88]:
from bs4 import BeautifulSoup
import requests
import time
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
from selenium import webdriver
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:13 http://ppa.launchpad.net/graph

In [89]:
!dpkg --configure -a
!apt-get install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

Reading package lists... Done
Building dependency tree       
Reading state information... Done
chromium-chromedriver is already the newest version (99.0.4844.51-0ubuntu0.18.04.1).
The following package was automatically installed and is no longer required:
  libnvidia-common-470
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 78 not upgraded.
cp: '/usr/lib/chromium-browser/chromedriver' and '/usr/bin/chromedriver' are the same file


In [90]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
# open it, go to a website, and get results
wd = webdriver.Chrome(options=options)
wd.get("https://www.gbif.org/occurrence/gallery?dataset_key=861e6afe-f762-11e1-a439-00145eb45e9a")
time.sleep(5)
html = wd.page_source
soup = BeautifulSoup(html, 'html.parser')

In [91]:
links = []
for sample in soup.find_all("a", class_="imageGallery__img ng-scope isValid"): # class for all specimen images on the HUH landing page
  links.append('https://www.gbif.org' + sample.get('href')) # returns every image with url to image src and specimen name
print(len(links)) # IMPORTANT: BROWSE to one of these links such as https://www.gbif.org/occurrence/2575024125 to see how we can get geography, taxon, and date too

100


In [92]:
imgs = []
names = []
for i in links:
  wd.get(i)
  time.sleep(5) # so the site admins don't think we are copying their site ;)
  html = wd.page_source
  soup = BeautifulSoup(html, 'html.parser')
  im = soup.find("img") # finds all images from main page 
  imgs.append(im['src']) # grabs source url per image
  nm = soup.find_all("span", class_="ng-binding")
  c = nm[12] # the 13th entry in span elements per page after languages is the specimen's name
  names.append(c.get_text()) # return text from span element = name of specimen
  
  #TODO: soup.find_all() where class_=taxon's class, then geography's class, then date's class, so we can store this information along with name, photo in matching order

for j in names:
  print(j)
for i in imgs:
  print(i)


Cleistes rosea Lindl.
Dicentra cucullaria (L.) Bernh.
Caltha palustris L.
Rubus fulleri L.H.Bailey
Halerpestes cymbalaria (Pursh) Greene
Ranunculus flammula L.
Chamaecrista fasciculata (Michx.) Greene
Persicaria robustior (Small) E.P.Bicknell
Juniperus chinensis L.
Planera aquatica J.F.Gmel.
Crataegus aestivalis (Walter) Torr. & A.Gray
Prunus umbellata Elliott
Oxalis brasiliensis Lodd. ex Hildebr.
Cardamine hirsuta L.
Oxalis articulata Savigny
Senecio vulgaris L.
Oxalis brasiliensis Lodd. ex Hildebr.
Viburnum nudum L.
Plantago pusilla Nutt.
Plantago maritima subsp. juncoides (Lam.) Hultén
Viburnum lentago L.
Viburnum opulus L.
Colutea arborescens L.
Carex crinita var. brevicrinis Fernald
Carex tonsa (Fernald) E.P.Bicknell
Alnus viridis (Chaix) DC.
Carex spicata Huds.
Carex deflexa Hornem.
Salix bebbiana Sarg.
Carex hirta L.
Pinus banksiana Lamb.
Fallopia ciliinodis (Michx.) Holub
Woodsia ilvensis (L.) R.Br.
Linum virginianum L.
Carex crinita var. brevicrinis Fernald
Selaginella rupestr

In [94]:
#names.remove("?") # one faulty element, yo beautifulsoup go crazy it learned to skip this element
#imgs.remove("/img/full_logo_white.svg") # one faulty element, same index as in names luckily, nvm bs4 skipped this 
print("There are:", len(names),"names in the current scrape of HUH")
print("There are:", len(imgs) ,"images in the current scrape of HUH")
!mkdir images # creates local directory for the images to be stored 

There are: 100 names in the current scrape of HUH
There are: 100 images in the current scrape of HUH
mkdir: cannot create directory ‘images’: File exists


In [95]:
import os
for i in range(len(imgs)):
  r = requests.get('http:' + imgs[i]).content
  time.sleep(2) # again to not attack the site with requests!
  #print(r.headers.get('content-type')) # requests.get().content converts object to bytes, remove .content and this line shows type of images = image/jpeg
  completeName = os.path.join('images', names[i] + str(i) + ".jpeg") # assigns proper type to images and combines path to images dir
  print(completeName)
  with open(completeName,"wb+") as f: # os's open done with write access, binary, + is create file if DNE, if exists, overwrite
        f.write(r)

images/Cleistes rosea Lindl.0.jpeg
images/Dicentra cucullaria (L.) Bernh.1.jpeg
images/Caltha palustris L.2.jpeg
images/Rubus fulleri L.H.Bailey3.jpeg
images/Halerpestes cymbalaria (Pursh) Greene4.jpeg
images/Ranunculus flammula L.5.jpeg
images/Chamaecrista fasciculata (Michx.) Greene6.jpeg
images/Persicaria robustior (Small) E.P.Bicknell7.jpeg
images/Juniperus chinensis L.8.jpeg
images/Planera aquatica J.F.Gmel.9.jpeg
images/Crataegus aestivalis (Walter) Torr. & A.Gray10.jpeg
images/Prunus umbellata Elliott11.jpeg
images/Oxalis brasiliensis Lodd. ex Hildebr.12.jpeg
images/Cardamine hirsuta L.13.jpeg
images/Oxalis articulata Savigny14.jpeg
images/Senecio vulgaris L.15.jpeg
images/Oxalis brasiliensis Lodd. ex Hildebr.16.jpeg
images/Viburnum nudum L.17.jpeg
images/Plantago pusilla Nutt.18.jpeg
images/Plantago maritima subsp. juncoides (Lam.) Hultén19.jpeg
images/Viburnum lentago L.20.jpeg
images/Viburnum opulus L.21.jpeg
images/Colutea arborescens L.22.jpeg
images/Carex crinita var. brev

In [108]:
for i in range(len(names)):
  with open("newcorpus.txt", "w") as f:
    for i in range(len(names)):
      f.write(names[i])
      f.write(' ')
for i in range(len(names)):
  with open("newgt.txt", "w") as f:
    for i in range(len(names)):
      f.write(names[i])
      f.write('\n')

In [109]:
!zip -r images/images.zip images/
!zip newcorpus.zip newcorpus.txt
!zip newgt.zip newgt.txt
from google.colab import files
files.download("images/images.zip")
files.download("newcorpus.zip")
files.download("newgt.zip")

updating: images/ (stored 0%)
updating: images/Coreopsis verticillata L.41.jpeg (deflated 2%)
updating: images/Juncus acuminatus Michx.78.jpeg (deflated 2%)
updating: images/Ficaria verna subsp. bulbifera Á. & D.Löve89.jpeg (deflated 2%)
updating: images/Silene antirrhina L.98.jpeg (deflated 2%)
updating: images/Polygala sanguinea L.83.jpeg (deflated 2%)
updating: images/Silene antirrhina L.94.jpeg (deflated 2%)
updating: images/Halerpestes cymbalaria (Pursh) Greene4.jpeg (deflated 3%)
updating: images/Spergularia marina (L.) Griseb.93.jpeg (deflated 2%)
updating: images/Kummerowia striata (Thunb.) Schindl.77.jpeg (deflated 2%)
updating: images/Carex hirta L.29.jpeg (deflated 2%)
updating: images/Ranunculus flammula L.5.jpeg (deflated 3%)
updating: images/Cyperus odoratus L.82.jpeg (deflated 2%)
updating: images/Carex crinita var. brevicrinis Fernald23.jpeg (deflated 2%)
updating: images/Spiranthes ochroleuca (Rydb.) Rydb.75.jpeg (deflated 3%)
updating: images/.ipynb_checkpoints/ (stor

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>